# 🔄 Hybrid AI Assistant Configuration

This notebook sets up a hybrid approach where you can:
- Keep your **frontend and API** running locally
- Use **Colab for model training** and heavy computations
- **Seamlessly switch** between local and cloud inference
- **Sync data and models** between environments

## 🏗️ Architecture Overview

```
Local Environment          │  Google Colab
                          │
┌─────────────────────┐    │  ┌─────────────────────┐
│   Frontend (React)  │    │  │  Model Training     │
│   - UI Components   │    │  │  - Data Processing  │
│   - User Interface  │    │  │  - Fine-tuning      │
└─────────────────────┘    │  │  - Evaluation       │
           │               │  └─────────────────────┘
┌─────────────────────┐    │  ┌─────────────────────┐
│   Backend API       │◄───┼──┤  Inference Service  │
│   - FastAPI         │    │  │  - Model Serving    │
│   - Local Models    │    │  │  - API Endpoints    │
│   - File System     │    │  │  - ngrok Tunnel     │
└─────────────────────┘    │  └─────────────────────┘
           │               │           │
┌─────────────────────┐    │  ┌─────────────────────┐
│   Google Drive      │◄───┼──┤  Model Storage      │
│   - Model Sync      │    │  │  - Trained Models   │
│   - Data Backup     │    │  │  - Checkpoints      │
└─────────────────────┘    │  └─────────────────────┘
```

## ✨ Benefits

- 🏠 **Local Development**: Keep your familiar development environment
- ☁️ **Cloud Training**: Leverage free GPU/TPU for model training
- 🔄 **Flexible Switching**: Use local or cloud models as needed
- 💰 **Cost Effective**: Only use cloud resources when necessary
- 🔒 **Data Control**: Keep sensitive data local when needed


In [ ]:
# 🔧 Setup Hybrid Environment

!pip install fastapi uvicorn requests python-multipart
!pip install google-colab-tools

from google.colab import drive, files
import os
import json
import requests
from pathlib import Path
import subprocess

# Mount Google Drive
drive.mount('/content/drive')

# Setup project paths
project_path = '/content/drive/MyDrive/ai-coding-assistant'
hybrid_path = f'{project_path}/hybrid'
config_path = f'{hybrid_path}/config'
models_path = f'{hybrid_path}/models'
sync_path = f'{hybrid_path}/sync'

for path in [hybrid_path, config_path, models_path, sync_path]:
    os.makedirs(path, exist_ok=True)

print('✅ Hybrid environment setup complete')
print(f'📁 Project path: {project_path}')
print(f'🔄 Hybrid path: {hybrid_path}')

In [ ]:
# ⚙️ Create Hybrid Configuration

hybrid_config = {
    "environment": {
        "local": {
            "frontend_url": "http://localhost:3000",
            "backend_url": "http://localhost:8000",
            "models_path": "./models",
            "data_path": "./data"
        },
        "colab": {
            "inference_url": "",  # Will be set when ngrok starts
            "models_path": "/content/drive/MyDrive/ai-coding-assistant/hybrid/models",
            "data_path": "/content/drive/MyDrive/ai-coding-assistant/hybrid/data",
            "gpu_available": True
        }
    },
    "models": {
        "primary": "local",  # local, colab, or openai
        "fallback": "openai",
        "local_models": [
            {
                "name": "custom_codellama",
                "path": "./models/fine_tuned_codellama",
                "type": "local",
                "capabilities": ["code_generation", "completion", "chat"]
            }
        ],
        "colab_models": [
            {
                "name": "colab_codellama",
                "path": "/content/drive/MyDrive/ai-coding-assistant/hybrid/models/fine_tuned_codellama",
                "type": "colab",
                "capabilities": ["code_generation", "completion", "chat", "training"]
            }
        ]
    },
    "sync": {
        "auto_sync": True,
        "sync_models": True,
        "sync_data": True,
        "sync_interval": 3600,  # 1 hour
        "compression": True
    },
    "features": {
        "load_balancing": True,
        "health_checks": True,
        "automatic_fallback": True,
        "performance_monitoring": True
    }
}

# Save configuration
config_file = f'{config_path}/hybrid_config.json'
with open(config_file, 'w') as f:
    json.dump(hybrid_config, f, indent=2)

print(f'✅ Hybrid configuration saved to: {config_file}')
print('📋 Configuration overview:')
print(f'  - Primary model source: {hybrid_config["models"]["primary"]}')
print(f'  - Fallback: {hybrid_config["models"]["fallback"]}')
print(f'  - Auto sync: {hybrid_config["sync"]["auto_sync"]}')
print(f'  - Load balancing: {hybrid_config["features"]["load_balancing"]}')

In [ ]:
# 🚀 Create Colab Inference Service

colab_service_code = '''
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import json
import logging
from typing import Optional, Dict, Any

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

app = FastAPI(title="Colab AI Inference Service", version="1.0.0")

# Request models
class CodeGenerationRequest(BaseModel):
    prompt: str
    language: str = "python"
    max_length: int = 512
    temperature: float = 0.7

class CompletionRequest(BaseModel):
    code: str
    cursor_position: int
    max_length: int = 256

class ChatRequest(BaseModel):
    message: str
    context: Optional[str] = None
    max_length: int = 512

# Global model variables
model = None
tokenizer = None
model_loaded = False

def load_model(model_path: str, base_model_name: str = "codellama/CodeLlama-7b-hf"):
    global model, tokenizer, model_loaded
    
    try:
        logger.info(f"Loading model from {model_path}")
        
        # Load tokenizer
        tokenizer = AutoTokenizer.from_pretrained(model_path)
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
        
        # Load base model
        base_model = AutoModelForCausalLM.from_pretrained(
            base_model_name,
            torch_dtype=torch.float16,
            device_map="auto"
        )
        
        # Load LoRA adapter
        model = PeftModel.from_pretrained(base_model, model_path)
        model_loaded = True
        
        logger.info("Model loaded successfully")
        
    except Exception as e:
        logger.error(f"Error loading model: {e}")
        model_loaded = False
        raise

def generate_text(prompt: str, max_length: int = 512, temperature: float = 0.7) -> str:
    if not model_loaded:
        raise HTTPException(status_code=503, detail="Model not loaded")
    
    try:
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_length=max_length,
                temperature=temperature,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id
            )
        
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        generated = response[len(prompt):].strip()
        
        return generated
        
    except Exception as e:
        logger.error(f"Error generating text: {e}")
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/")
async def root():
    return {
        "service": "Colab AI Inference Service",
        "status": "running",
        "model_loaded": model_loaded,
        "gpu_available": torch.cuda.is_available()
    }

@app.get("/health")
async def health_check():
    return {
        "status": "healthy",
        "model_loaded": model_loaded,
        "gpu_memory": torch.cuda.get_device_properties(0).total_memory / 1e9 if torch.cuda.is_available() else 0
    }

@app.post("/generate-code")
async def generate_code(request: CodeGenerationRequest):
    formatted_prompt = f"### Instruction: Generate {request.language} code for: {request.prompt}\n### Input: \n### Output: "
    
    generated = generate_text(
        formatted_prompt,
        max_length=request.max_length,
        temperature=request.temperature
    )
    
    return {"generated_code": generated}

@app.post("/suggest-completion")
async def suggest_completion(request: CompletionRequest):
    formatted_prompt = f"### Instruction: Complete this code\n### Input: {request.code}\n### Output: "
    
    generated = generate_text(
        formatted_prompt,
        max_length=request.max_length,
        temperature=0.3
    )
    
    return {"completion": generated}

@app.post("/chat")
async def chat(request: ChatRequest):
    formatted_prompt = f"### Instruction: Answer this programming question\n### Input: {request.message}\n### Output: "
    
    generated = generate_text(
        formatted_prompt,
        max_length=request.max_length,
        temperature=0.7
    )
    
    return {"response": generated}

@app.post("/load-model")
async def load_model_endpoint(model_path: str, base_model: str = "codellama/CodeLlama-7b-hf"):
    try:
        load_model(model_path, base_model)
        return {"status": "success", "message": "Model loaded successfully"}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8080)
'''

# Save the Colab inference service
service_file = f'{hybrid_path}/colab_inference_service.py'
with open(service_file, 'w') as f:
    f.write(colab_service_code.strip())

print(f'✅ Colab inference service created: {service_file}')

In [ ]:
# 🏠 Create Local Hybrid Service

local_hybrid_service = '''
import asyncio
import aiohttp
import json
import logging
from typing import Optional, Dict, Any, List
from pathlib import Path
import time

logger = logging.getLogger(__name__)

class HybridAIService:
    def __init__(self, config_path: str = "./hybrid/config/hybrid_config.json"):
        self.config_path = config_path
        self.config = self.load_config()
        self.local_service = None
        self.colab_url = None
        self.health_status = {
            "local": False,
            "colab": False,
            "openai": True  # Assume OpenAI is available
        }
        
        # Initialize local service if available
        self.init_local_service()
    
    def load_config(self) -> Dict[str, Any]:
        """Load hybrid configuration"""
        try:
            with open(self.config_path, 'r') as f:
                return json.load(f)
        except Exception as e:
            logger.error(f"Error loading config: {e}")
            return self.get_default_config()
    
    def get_default_config(self) -> Dict[str, Any]:
        """Get default configuration"""
        return {
            "models": {
                "primary": "openai",
                "fallback": "openai"
            },
            "features": {
                "automatic_fallback": True,
                "health_checks": True
            }
        }
    
    def init_local_service(self):
        """Initialize local model service if available"""
        try:
            # Try to import and initialize local model service
            from .custom_model_service import CustomModelService
            
            local_models = self.config.get("models", {}).get("local_models", [])
            if local_models:
                model_path = local_models[0]["path"]
                if Path(model_path).exists():
                    self.local_service = CustomModelService(model_path)
                    self.health_status["local"] = True
                    logger.info("Local model service initialized")
                
        except Exception as e:
            logger.warning(f"Local model service not available: {e}")
            self.health_status["local"] = False
    
    async def check_colab_health(self) -> bool:
        """Check if Colab service is available"""
        if not self.colab_url:
            return False
        
        try:
            async with aiohttp.ClientSession() as session:
                async with session.get(f"{self.colab_url}/health", timeout=5) as response:
                    if response.status == 200:
                        data = await response.json()
                        return data.get("status") == "healthy"
        except Exception as e:
            logger.warning(f"Colab health check failed: {e}")
        
        return False
    
    async def update_health_status(self):
        """Update health status of all services"""
        if self.config.get("features", {}).get("health_checks", True):
            self.health_status["colab"] = await self.check_colab_health()
    
    def get_best_service(self, task_type: str = "general") -> str:
        """Determine the best service to use based on availability and configuration"""
        primary = self.config.get("models", {}).get("primary", "openai")
        fallback = self.config.get("models", {}).get("fallback", "openai")
        
        # Check primary service
        if primary == "local" and self.health_status["local"]:
            return "local"
        elif primary == "colab" and self.health_status["colab"]:
            return "colab"
        elif primary == "openai":
            return "openai"
        
        # Fallback to available service
        if self.config.get("features", {}).get("automatic_fallback", True):
            if fallback == "local" and self.health_status["local"]:
                return "local"
            elif fallback == "colab" and self.health_status["colab"]:
                return "colab"
            else:
                return "openai"
        
        return "openai"  # Final fallback
    
    async def generate_code(self, prompt: str, language: str = "python") -> str:
        """Generate code using the best available service"""
        await self.update_health_status()
        service = self.get_best_service("code_generation")
        
        logger.info(f"Using {service} service for code generation")
        
        try:
            if service == "local" and self.local_service:
                return await self.local_service.generate_code(prompt, language)
            
            elif service == "colab" and self.colab_url:
                return await self._call_colab_service("/generate-code", {
                    "prompt": prompt,
                    "language": language
                })
            
            else:
                # Fallback to OpenAI
                return await self._call_openai_service("generate_code", prompt, language)
                
        except Exception as e:
            logger.error(f"Error in {service} service: {e}")
            # Try fallback
            if service != "openai":
                return await self._call_openai_service("generate_code", prompt, language)
            raise
    
    async def suggest_completion(self, code: str, cursor_position: int) -> str:
        """Suggest code completion using the best available service"""
        await self.update_health_status()
        service = self.get_best_service("completion")
        
        logger.info(f"Using {service} service for code completion")
        
        try:
            if service == "local" and self.local_service:
                return await self.local_service.suggest_completion(code, cursor_position)
            
            elif service == "colab" and self.colab_url:
                return await self._call_colab_service("/suggest-completion", {
                    "code": code,
                    "cursor_position": cursor_position
                })
            
            else:
                return await self._call_openai_service("suggest_completion", code, cursor_position)
                
        except Exception as e:
            logger.error(f"Error in {service} service: {e}")
            if service != "openai":
                return await self._call_openai_service("suggest_completion", code, cursor_position)
            raise
    
    async def chat_with_context(self, message: str, context=None) -> str:
        """Chat with context using the best available service"""
        await self.update_health_status()
        service = self.get_best_service("chat")
        
        logger.info(f"Using {service} service for chat")
        
        try:
            if service == "local" and self.local_service:
                return await self.local_service.chat_with_context(message, context)
            
            elif service == "colab" and self.colab_url:
                return await self._call_colab_service("/chat", {
                    "message": message,
                    "context": context
                })
            
            else:
                return await self._call_openai_service("chat_with_context", message, context)
                
        except Exception as e:
            logger.error(f"Error in {service} service: {e}")
            if service != "openai":
                return await self._call_openai_service("chat_with_context", message, context)
            raise
    
    async def _call_colab_service(self, endpoint: str, data: Dict[str, Any]) -> str:
        """Call Colab inference service"""
        async with aiohttp.ClientSession() as session:
            async with session.post(f"{self.colab_url}{endpoint}", json=data) as response:
                if response.status == 200:
                    result = await response.json()
                    return result.get("generated_code") or result.get("completion") or result.get("response", "")
                else:
                    raise Exception(f"Colab service error: {response.status}")
    
    async def _call_openai_service(self, method: str, *args, **kwargs) -> str:
        """Fallback to OpenAI service"""
        # Import your existing OpenAI service
        from .ai_service import AIService
        
        ai_service = AIService()
        
        if method == "generate_code":
            return await ai_service.generate_code(*args, **kwargs)
        elif method == "suggest_completion":
            return await ai_service.suggest_completion(*args, **kwargs)
        elif method == "chat_with_context":
            return await ai_service.chat_with_context(*args, **kwargs)
        else:
            raise ValueError(f"Unknown method: {method}")
    
    def set_colab_url(self, url: str):
        """Set the Colab service URL"""
        self.colab_url = url
        logger.info(f"Colab URL set to: {url}")
    
    def get_status(self) -> Dict[str, Any]:
        """Get current status of all services"""
        return {
            "health_status": self.health_status,
            "primary_service": self.config.get("models", {}).get("primary", "openai"),
            "colab_url": self.colab_url,
            "config_loaded": bool(self.config)
        }
'''

# Save the local hybrid service
local_service_file = f'{hybrid_path}/hybrid_ai_service.py'
with open(local_service_file, 'w') as f:
    f.write(local_hybrid_service.strip())

print(f'✅ Local hybrid service created: {local_service_file}')

In [ ]:
# 🌐 Setup ngrok Tunnel for Colab Service

!pip install pyngrok

from pyngrok import ngrok
import threading
import uvicorn
import sys
import os

# Set ngrok auth token
ngrok.set_auth_token("32lxmVvlcVwe0aIQldrGTv9sB0c_2kNsr5fcetExZj6St12dZ")

# Add the hybrid path to Python path
sys.path.append(hybrid_path)

def start_inference_service():
    """Start the inference service in a separate thread"""
    # Import the service
    from colab_inference_service import app, load_model
    
    # Load a model if available
    model_path = f'{models_path}/fine_tuned_codellama'
    if os.path.exists(model_path):
        try:
            load_model(model_path)
            print(f'✅ Model loaded from {model_path}')
        except Exception as e:
            print(f'⚠️ Could not load model: {e}')
    else:
        print('⚠️ No trained model found. Service will run without model.')
    
    # Start the service
    uvicorn.run(app, host="0.0.0.0", port=8080, log_level="info")

def setup_ngrok_tunnel():
    """Setup ngrok tunnel for the inference service"""
    try:
        # Kill any existing ngrok processes
        ngrok.kill()
        
        # Create tunnel
        public_url = ngrok.connect(8080)
        print(f'🌐 ngrok tunnel created: {public_url}')
        
        # Update configuration with the public URL
        hybrid_config['environment']['colab']['inference_url'] = str(public_url)
        
        with open(config_file, 'w') as f:
            json.dump(hybrid_config, f, indent=2)
        
        print(f'✅ Configuration updated with ngrok URL')
        print(f'🔗 Your Colab AI service is available at: {public_url}')
        
        return str(public_url)
        
    except Exception as e:
        print(f'❌ Error setting up ngrok: {e}')
        return None

# Start the inference service in background
print('🚀 Starting Colab inference service...')
service_thread = threading.Thread(target=start_inference_service, daemon=True)
service_thread.start()

# Wait a moment for the service to start
import time
time.sleep(5)

# Setup ngrok tunnel
public_url = setup_ngrok_tunnel()

if public_url:
    print('\n🎉 Hybrid setup complete!')
    print(f'📡 Colab service URL: {public_url}')
    print('📋 Next steps:')
    print('  1. Copy the hybrid configuration to your local project')
    print('  2. Update your local AI service to use HybridAIService')
    print('  3. Set the Colab URL in your local configuration')
    print('  4. Test the hybrid functionality')
else:
    print('❌ Failed to setup ngrok tunnel. Service running locally only.')

In [ ]:
# 🔄 Create Sync Utilities

sync_utilities = '''
import os
import shutil
import json
import zipfile
import requests
from pathlib import Path
from typing import Dict, List, Optional
import logging
from datetime import datetime

logger = logging.getLogger(__name__)

class HybridSyncManager:
    def __init__(self, config_path: str):
        self.config_path = config_path
        self.config = self.load_config()
        self.sync_log = []
    
    def load_config(self) -> Dict:
        with open(self.config_path, 'r') as f:
            return json.load(f)
    
    def sync_models_to_local(self, colab_models_path: str, local_models_path: str):
        """Sync models from Colab to local environment"""
        try:
            logger.info(f"Syncing models from {colab_models_path} to {local_models_path}")
            
            # Create local models directory
            os.makedirs(local_models_path, exist_ok=True)
            
            # Copy model files
            if os.path.exists(colab_models_path):
                for item in os.listdir(colab_models_path):
                    src = os.path.join(colab_models_path, item)
                    dst = os.path.join(local_models_path, item)
                    
                    if os.path.isdir(src):
                        shutil.copytree(src, dst, dirs_exist_ok=True)
                    else:
                        shutil.copy2(src, dst)
                
                self.log_sync("models", "colab_to_local", "success")
                logger.info("Models synced successfully")
            else:
                logger.warning(f"Colab models path not found: {colab_models_path}")
                
        except Exception as e:
            logger.error(f"Error syncing models: {e}")
            self.log_sync("models", "colab_to_local", "error", str(e))
    
    def sync_data_to_colab(self, local_data_path: str, colab_data_path: str):
        """Sync data from local to Colab environment"""
        try:
            logger.info(f"Syncing data from {local_data_path} to {colab_data_path}")
            
            # Create colab data directory
            os.makedirs(colab_data_path, exist_ok=True)
            
            # Copy data files
            if os.path.exists(local_data_path):
                for item in os.listdir(local_data_path):
                    src = os.path.join(local_data_path, item)
                    dst = os.path.join(colab_data_path, item)
                    
                    if os.path.isdir(src):
                        shutil.copytree(src, dst, dirs_exist_ok=True)
                    else:
                        shutil.copy2(src, dst)
                
                self.log_sync("data", "local_to_colab", "success")
                logger.info("Data synced successfully")
            else:
                logger.warning(f"Local data path not found: {local_data_path}")
                
        except Exception as e:
            logger.error(f"Error syncing data: {e}")
            self.log_sync("data", "local_to_colab", "error", str(e))
    
    def create_backup(self, source_path: str, backup_path: str, compress: bool = True):
        """Create backup of models or data"""
        try:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            
            if compress:
                backup_file = f"{backup_path}/backup_{timestamp}.zip"
                os.makedirs(os.path.dirname(backup_file), exist_ok=True)
                
                with zipfile.ZipFile(backup_file, 'w', zipfile.ZIP_DEFLATED) as zipf:
                    for root, dirs, files in os.walk(source_path):
                        for file in files:
                            file_path = os.path.join(root, file)
                            arcname = os.path.relpath(file_path, source_path)
                            zipf.write(file_path, arcname)
                
                logger.info(f"Backup created: {backup_file}")
                return backup_file
            else:
                backup_dir = f"{backup_path}/backup_{timestamp}"
                shutil.copytree(source_path, backup_dir)
                logger.info(f"Backup created: {backup_dir}")
                return backup_dir
                
        except Exception as e:
            logger.error(f"Error creating backup: {e}")
            return None
    
    def log_sync(self, sync_type: str, direction: str, status: str, error: str = None):
        """Log sync operation"""
        log_entry = {
            "timestamp": datetime.now().isoformat(),
            "type": sync_type,
            "direction": direction,
            "status": status,
            "error": error
        }
        
        self.sync_log.append(log_entry)
    
    def get_sync_history(self) -> List[Dict]:
        """Get sync history"""
        return self.sync_log
    
    def save_sync_log(self, log_path: str):
        """Save sync log to file"""
        with open(log_path, 'w') as f:
            json.dump(self.sync_log, f, indent=2)

# Utility functions for easy use
def quick_sync_models(config_path: str):
    """Quick function to sync models from Colab to local"""
    sync_manager = HybridSyncManager(config_path)
    config = sync_manager.config
    
    colab_path = config['environment']['colab']['models_path']
    local_path = config['environment']['local']['models_path']
    
    sync_manager.sync_models_to_local(colab_path, local_path)
    return sync_manager.get_sync_history()

def quick_sync_data(config_path: str):
    """Quick function to sync data from local to Colab"""
    sync_manager = HybridSyncManager(config_path)
    config = sync_manager.config
    
    local_path = config['environment']['local']['data_path']
    colab_path = config['environment']['colab']['data_path']
    
    sync_manager.sync_data_to_colab(local_path, colab_path)
    return sync_manager.get_sync_history()
'''

# Save sync utilities
sync_file = f'{hybrid_path}/sync_utilities.py'
with open(sync_file, 'w') as f:
    f.write(sync_utilities.strip())

print(f'✅ Sync utilities created: {sync_file}')

In [ ]:
# 📖 Create Integration Guide

integration_guide = '''
# 🔄 Hybrid AI Assistant Integration Guide

This guide helps you integrate the hybrid approach with your existing AI coding assistant.

## 📁 File Structure

```
ai-coding-assistant/
├── backend/
│   ├── app/
│   │   ├── services/
│   │   │   ├── ai_service.py          # Original AI service
│   │   │   ├── hybrid_ai_service.py   # New hybrid service
│   │   │   └── custom_model_service.py # Local model service
│   │   └── main.py
├── frontend/
├── hybrid/
│   ├── config/
│   │   └── hybrid_config.json         # Hybrid configuration
│   ├── models/                        # Local model storage
│   ├── colab_inference_service.py     # Colab service
│   ├── hybrid_ai_service.py           # Hybrid service
│   └── sync_utilities.py              # Sync utilities
└── colab_notebooks/
    ├── 01_AI_Assistant_Migration.ipynb
    ├── 02_Custom_Model_Training.ipynb
    └── 03_Hybrid_Configuration.ipynb
```

## 🔧 Integration Steps

### Step 1: Copy Hybrid Files to Local Project

1. Download the hybrid folder from Google Drive
2. Copy it to your local project root
3. Copy `hybrid_ai_service.py` to `backend/app/services/`

### Step 2: Update Your Backend Service

Replace your existing AI service import in `main.py`:

```python
# OLD
from app.services.ai_service import AIService

# NEW
from app.services.hybrid_ai_service import HybridAIService

# Initialize hybrid service
ai_service = HybridAIService()
```

### Step 3: Configure Colab URL

Set the Colab service URL in your backend:

```python
# Set Colab URL (from ngrok output above)
ai_service.set_colab_url('https://your-ngrok-url.ngrok.io')
```

### Step 4: Test Integration

Test your hybrid setup:

```python
# Test code generation
result = await ai_service.generate_code('create a function to sort a list', 'python')
print(result)

# Check service status
status = ai_service.get_status()
print(status)
```

**Congratulations! Your hybrid AI assistant is now configured! 🎉**
'''

# Save integration guide
guide_file = f'{hybrid_path}/INTEGRATION_GUIDE.md'
with open(guide_file, 'w') as f:
    f.write(integration_guide.strip())

print(f'✅ Integration guide created: {guide_file}')
print('\n🎊 Hybrid configuration complete!')
print('📋 Summary of created files:')
print(f'  - Configuration: {config_file}')
print(f'  - Colab Service: {service_file}')
print(f'  - Local Service: {local_service_file}')
print(f'  - Sync Utilities: {sync_file}')
print(f'  - Integration Guide: {guide_file}')
print('\n🚀 Next steps:')
print('  1. Follow the integration guide to set up your local environment')
print('  2. Test the hybrid functionality')
print('  3. Train custom models using the training notebook')
print('  4. Enjoy your hybrid AI coding assistant!')

## 🎉 Hybrid Configuration Complete!

Your hybrid AI coding assistant setup is now ready! This configuration allows you to:

### ✨ Key Features

- **🏠 Local Development**: Keep your frontend and backend running locally
- **☁️ Cloud Training**: Use Colab's free GPU for model training
- **🔄 Automatic Fallback**: Seamlessly switch between services
- **📊 Load Balancing**: Distribute requests across available services
- **🔄 Easy Sync**: Keep models and data synchronized

### 📁 Files Created

1. **hybrid_config.json** - Main configuration file
2. **colab_inference_service.py** - Colab inference service
3. **hybrid_ai_service.py** - Local hybrid service manager
4. **sync_utilities.py** - Data and model synchronization
5. **INTEGRATION_GUIDE.md** - Complete integration instructions

### 🎯 Next Steps

1. **Download Files**: Download the hybrid folder from Google Drive
2. **Follow Guide**: Use the integration guide to set up locally
3. **Test Setup**: Verify all services work correctly
4. **Train Models**: Use the training notebook for custom models

### 🚀 Benefits of This Setup

- **Cost Effective**: Only pay for cloud resources when needed
- **High Performance**: Use the best service for each task
- **Reliable**: Multiple fallback options ensure uptime
- **Scalable**: Easy to add more services or models
- **Flexible**: Switch between local and cloud as needed

**Your hybrid AI coding assistant is ready to use! 🎊**
